# Calibration Factor Estimation with `Stan`

In [4]:
import sys
import numpy as np
import scipy.special
import pystan 
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../../')
import mwc.viz
import mwc.bayes
import mwc.stats
import bokeh.io
import bokeh.plotting
colors = mwc.viz.personal_style()
bokeh.io.output_notebook()
%matplotlib inline

Loading BokehJS ...

In [5]:
# Define the constants
n_div = 100
alpha_true = 200
alpha_sd = 20

# Simulate the data
alpha_dist_1 = np.random.normal(loc=alpha_true, scale=alpha_sd, size=n_div)
alpha_dist_2 = np.random.normal(loc=alpha_true, scale=alpha_sd, size=n_div)
ntot = np.random.gamma(5, 25, size=n_div).astype(int)
n1 = np.random.binomial(ntot, 0.5)
n2 = ntot - n1
I1 = n1 * alpha_dist_1
I2 = n2 * alpha_dist_2

In [6]:
# Compute the sqare difference and the sum total
sq_diff = (I1 - I2)**2
summed = I1 + I2

p = bokeh.plotting.figure(plot_width=600, plot_height=400, x_axis_type='log', y_axis_type='log',
                          x_axis_label='summed intensity', y_axis_label = 'squared difference')
p.circle(summed, sq_diff, color=colors[0], alpha=0.5)
bokeh.io.show(p)

In [25]:
# Define the stan model.
model_code = """
data {
    int<lower=1> N; // Number of data points.
    vector<lower=0>[N] I1; // Intensity of daughter 1
    vector<lower=0>[N] I2; // Intensity of daughter 2
    real<lower=0, upper=1> p;
    }

parameters {
    real<lower=0> alpha; // Calibration factor. 
    real<lower=1E-9> sigma; // Homoscedastic error
    vector<lower=1>[N] N_tot; // Total number of proteins
    vector<lower=0, upper=N_tot[N]>[N] N1; // proteins in daugter 1
    }

model {
   alpha ~ normal(0, 1000);
   N_tot ~ normal(0, 100);
   sigma ~ normal(0, 10);
   N1 ~ normal(N_tot * p, sqrt(N_tot * p * (1 - p)));
   I1 ~ normal(alpha * N1, sigma);
   I2 ~ normal(alpha * (N_tot - N1), sigma);
   }
"""
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_98cc26ae08c165a1fcc9d510834e3cf6 NOW.


In [ ]:
# Define the dictionary
data_dict = {'N':n_div, 'I1':I1, 'I2':I2, 'p':0.5}

# Sample the distribution
samples = model.sampling(data=data_dict, iter=100000, chains=48, thin=100)

In [27]:
samples

Inference for Stan model: anon_model_98cc26ae08c165a1fcc9d510834e3cf6.
48 chains, each with iter=100000; warmup=50000; thin=100; 
post-warmup draws per chain=500, total post-warmup draws=24000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     341.86   27.96 136.98 263.03 289.94 303.22 331.05 785.05     24  14.91
sigma       3.99    0.76   4.55    0.1   0.76   2.19    5.7  16.33     36    1.9
N_tot[0]   82.13    3.02  15.09  33.33  79.04   86.3  90.25  99.49     25   6.16
N_tot[1]   41.63    1.53   7.65   16.9  40.07  43.75  45.75  50.43     25   6.16
N_tot[2]   51.53    1.89   9.47  20.91   49.6  54.15  56.64  62.43     25   6.16
N_tot[3]  150.58    5.53  27.66  61.12 144.93 158.23 165.48 182.42     25   6.16
N_tot[4]   40.88     1.5   7.51  16.59  39.34  42.96  44.93  49.52     25   6.16
N_tot[5]   81.66     3.0   15.0  33.14  78.59   85.8  89.73  98.92     25   6.16
N_tot[6]   53.36    1.96    9.8  21.66  51.36  56.07  58.64  64.64     25   